## Import niezbędnych modułów

In [1]:
import base64
import hashlib
import os
import random
import string

import pandas as pd
pd.set_option('display.width', 1000)

## Definicja własnych funkcji anonymizacji z wykorzystaniem biblioteki Pandas

In [23]:
def encode_column_random(data, column):
    random_values = data[column].apply(lambda row: os.urandom(12))
    data[column] = random_values.apply(lambda row: base64.b64encode(row).decode())
    return data

def encode_column_deterministic(data, column, token):
    hashed_data = data[column].apply(lambda row: hashlib.sha256(row.encode() + token.encode()))
    data[column] = hashed_data.apply(lambda row: base64.b64encode(row.digest()).decode()[0:16])
    return data

def swap_rows(data, column):
    shuffled_column = pd.DataFrame(data[column].sample(frac=1).reset_index(drop=True))
    data[column] = shuffled_column.set_index(data.index)
    return data

def mask_column(data, column, visible_chars_count=3, mask_whole_above=10, reverse=False):
    data[column] = data[column].apply(lambda row: (row[:visible_chars_count] + '****' if not reverse else '****' + row[-visible_chars_count:]) if len(row) > mask_whole_above else '****')
    return data

def _round_to_nearest(x, base):
    return base * round(x / base)

def perturbate_numerical_column(data, column, rounding, max_noise):
    data[column] = _round_to_nearest(data[column] + random.randint(0, max_noise) * random.choice([1,-1]), rounding)
    return data

## Załadowanie surowych danych

In [3]:
# Load the raw data
data_raw = pd.read_csv(os.path.join('data', 'fictitious_cutsomer_data.csv'), index_col='id')
print(data_raw)

     first_name      last_name         voivodeship       city                            email   phone_number  age  last_purchase_category  purchases_value
id                                                                                                                                                         
1         Filip       Adamczyk        Dolnośląskie    Wrocław       jacek.szymański58@mail.com  +48 807662339   21                       5              280
2      Grzegorz         Kaczor        Podkarpackie     Gdańsk     henryk.woźniak27@example.com  +48 050643694   20                       2              390
3         Beata      Dąbrowski  Kujawsko-Pomorskie     Lublin  grażyna.dąbrowski59@example.com  +48 555276613   43                       4              670
4        Danuta      Kaczmarek            Lubuskie   Warszawa     grażyna.dąbrowski28@mail.com  +48 267952410   24                       4              340
5        Łukasz        Woźniak             Łódzkie     Poznań   

# Anonimizacja

## 1. Pseudonimizacja (kodowanie) kolumn "first_name" i "last_name" oraz zastąpienie ich pojedycznym atrybutem "name"

In [26]:
data = data_raw
# concatenate the first_name and last_name columns
data['name'] = data[['first_name', 'last_name']].apply(lambda row: (row['first_name'] + row['last_name']), axis=1)
# Load the token used for hashing
token = None
with open('./token.txt', 'r') as file:
    token = file.read()
# encoded the selected column
data = encode_column_deterministic(data, column='name', token=token)
# drop the original columns
data = data.drop(['first_name', 'last_name'], axis=1)
# move the 'name' column to the front
data = data[['name'] + [col for col in data.columns if col != 'name']]
# print the resulting dataframe
print(data)

                 name         voivodeship       city                            email   phone_number  age  last_purchase_category  purchases_value
id                                                                                                                                                
1    K7MSlCevOa2KchxH        Dolnośląskie    Wrocław       jacek.szymański58@mail.com  +48 807662339   21                       5              280
2    +1d7RlVzv60m9kCZ        Podkarpackie     Gdańsk     henryk.woźniak27@example.com  +48 050643694   20                       2              390
3    klLAFhx8GedE3Moi  Kujawsko-Pomorskie     Lublin  grażyna.dąbrowski59@example.com  +48 555276613   43                       4              670
4    7DZT4hT7ZU4ojm3e            Lubuskie   Warszawa     grażyna.dąbrowski28@mail.com  +48 267952410   24                       4              340
5    gzGFTGx9obr6LwkZ             Łódzkie     Poznań   dariusz.kowalski24@example.com  +48 797852810   32             

## 2. Supresja kolumny "voivodeship"

In [27]:
# Supresja
data = data.drop('voivodeship', axis=1)
print(data)

                 name       city                            email   phone_number  age  last_purchase_category  purchases_value
id                                                                                                                            
1    K7MSlCevOa2KchxH    Wrocław       jacek.szymański58@mail.com  +48 807662339   21                       5              280
2    +1d7RlVzv60m9kCZ     Gdańsk     henryk.woźniak27@example.com  +48 050643694   20                       2              390
3    klLAFhx8GedE3Moi     Lublin  grażyna.dąbrowski59@example.com  +48 555276613   43                       4              670
4    7DZT4hT7ZU4ojm3e   Warszawa     grażyna.dąbrowski28@mail.com  +48 267952410   24                       4              340
5    gzGFTGx9obr6LwkZ     Poznań   dariusz.kowalski24@example.com  +48 797852810   32                       3              360
..                ...        ...                              ...            ...  ...                     ...  

## 3. Perturbacja danych w kolumnach "age" oraz "purchase_value"

In [28]:
# Perturbacja danych
data = perturbate_numerical_column(data, 'age', 5, 2)
data = perturbate_numerical_column(data, 'purchases_value', 50, 20)
print(data)

                 name       city                            email   phone_number   age  last_purchase_category  purchases_value
id                                                                                                                             
1    K7MSlCevOa2KchxH    Wrocław       jacek.szymański58@mail.com  +48 807662339  20.0                       5            300.0
2    +1d7RlVzv60m9kCZ     Gdańsk     henryk.woźniak27@example.com  +48 050643694  20.0                       2            400.0
3    klLAFhx8GedE3Moi     Lublin  grażyna.dąbrowski59@example.com  +48 555276613  45.0                       4            700.0
4    7DZT4hT7ZU4ojm3e   Warszawa     grażyna.dąbrowski28@mail.com  +48 267952410  25.0                       4            350.0
5    gzGFTGx9obr6LwkZ     Poznań   dariusz.kowalski24@example.com  +48 797852810  35.0                       3            350.0
..                ...        ...                              ...            ...   ...                  

## 4. Maskowanie znaków w kolumnach "email" oraz "phone_number"

In [29]:
# Maskowanie znaków
data = mask_column(data, 'email')
data = mask_column(data, 'phone_number', visible_chars_count=2, reverse=True)
print(data)

                 name       city    email phone_number   age  last_purchase_category  purchases_value
id                                                                                                   
1    K7MSlCevOa2KchxH    Wrocław  jac****       ****39  20.0                       5            300.0
2    +1d7RlVzv60m9kCZ     Gdańsk  hen****       ****94  20.0                       2            400.0
3    klLAFhx8GedE3Moi     Lublin  gra****       ****13  45.0                       4            700.0
4    7DZT4hT7ZU4ojm3e   Warszawa  gra****       ****10  25.0                       4            350.0
5    gzGFTGx9obr6LwkZ     Poznań  dar****       ****10  35.0                       3            350.0
..                ...        ...      ...          ...   ...                     ...              ...
96   WlCu8feROWJI714z  Bydgoszcz  bar****       ****19  20.0                       2             50.0
97   2T8LFx3WlkVX01dH     Gdańsk  iwo****       ****46  35.0                      

## 5. Permutacje (swapping) kolumny "city"

In [30]:
# Permutacje (swapping)
data = swap_rows(data, 'city')
print(data)

                 name       city    email phone_number   age  last_purchase_category  purchases_value
id                                                                                                   
1    K7MSlCevOa2KchxH  Bydgoszcz  jac****       ****39  20.0                       5            300.0
2    +1d7RlVzv60m9kCZ    Wrocław  hen****       ****94  20.0                       2            400.0
3    klLAFhx8GedE3Moi   Warszawa  gra****       ****13  45.0                       4            700.0
4    7DZT4hT7ZU4ojm3e   Warszawa  gra****       ****10  25.0                       4            350.0
5    gzGFTGx9obr6LwkZ   Warszawa  dar****       ****10  35.0                       3            350.0
..                ...        ...      ...          ...   ...                     ...              ...
96   WlCu8feROWJI714z     Gdańsk  bar****       ****19  20.0                       2             50.0
97   2T8LFx3WlkVX01dH     Poznań  iwo****       ****46  35.0                      

## 6. Agregacja danych

### 6.1 Obliczenie sumy kolumny "purchases_value" dla poszczególnych województw

In [31]:
# Agregacja danych I
result = data_raw[['voivodeship','purchases_value']].groupby('voivodeship').sum()
print(result)

                    purchases_value
voivodeship                        
Dolnośląskie                   3760
Kujawsko-Pomorskie             5210
Lubelskie                      7310
Lubuskie                      14950
Mazowieckie                   14660
Małopolskie                    6640
Opolskie                      21930
Podkarpackie                  17300
Podlaskie                     14440
Łódzkie                        8990


### 6.2 Sumowanie całej kolumny "purchases_value" 

In [32]:
# Agregacja danych II
result = data_raw['purchases_value'].sum(axis=0)
print(f'Total income: {result} PLN')

Total income: 115190 PLN
